In [122]:
import sys
sys.path.insert(0, '../..')
from helper import _process_pred_labels
from evaluate import get_pred_labels

In [123]:
import os
import pandas as pd
import numpy as np

In [139]:
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [142]:
N_FILES = 2

In [125]:
target = pd.read_pickle('../../data/prepared/target.pkl')

In [146]:
a = pd.DataFrame({'A': [1,3,2,5], 'B': [0.1, 0.5, 0.9, 0.3]})

In [147]:
a.rank(ascending=False)

,A,B
0,4.0,4.0
1,2.0,2.0
2,3.0,1.0
3,1.0,3.0


In [148]:
train = []
for i in range(N_FILES):
    df = pd.read_csv(f'oof_{i}/pred_proba.csv', index_col='review_id')
    df.columns = [f'{i}_{col}' for col in df.columns]
    df = df.rank(ascending=False) / len(df)
    train.append(df)
train = pd.concat(train, axis=1)

In [149]:
test = []
for i in range(N_FILES):
    df = pd.read_csv(f'sub_{i}/pred_proba.csv', index_col='review_id')
    df.columns = [f'{i}_{col}' for col in df.columns]
    df = df.rank(ascending=False) / len(df)
    test.append(df)
test = pd.concat(test, axis=1)

In [152]:
base_model = LogisticRegression()
model = MultiOutputClassifier(base_model, n_jobs=-1)

param_grid = {'estimator__base_estimator__C': np.random.uniform(low=0.1, high=5, size=20),
              'estimator__base_estimator__penalty': ['l2', 'l1'],
              'estimator__base_estimator__solver': ['liblinear'],
              'estimator__base_estimator__max_iter': [1000]}
gscv = GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1_samples', cv=5, n_jobs=1, verbose=5, error_score='raise')
gscv.fit(train, target)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


ValueError: Invalid parameter base_estimator for estimator LogisticRegression(). Check the list of available parameters with `estimator.get_params().keys()`.

In [151]:
gscv.best_score_

0.8319744505869506

In [ ]:
# 0.8432528292228764 - LR
# 0.8441071697984434 - LinSVC
# 0.8441432080880193 - LinSVC

In [120]:
pred_proba = gscv.predict_proba(test)
pred_labels = gscv.predict(test)

In [121]:
pred_proba = np.stack([p[:, 1] for p in pred_proba]).T
pred_proba = pd.DataFrame(pred_proba, index=test.index, columns=target.columns)
pred_proba = get_pred_labels(pred_proba)
pred_proba['target'] = pred_proba.apply(_process_pred_labels, axis=1)

TypeError: '(slice(None, None, None), 0)' is an invalid key

In [ ]:
pred_proba.to_csv('stacking3.csv')